In [12]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

In [2]:
df = pd.read_csv("DataResults/final.csv")
df = df.iloc[: , 1:]
df

,PID,Starting Interruption,Starting Task,Condition,Control,d_age,d_gender,d_education,d_asd,d_colorblind,...,te_p_count,te_p_correctness,te_p_time,te_p_times,te_p_percentage,te_p_percentage100,te_p_resumption,te_p_resumptions,te_p_interruptions,te_p_movestotal
0,150657-543833,1,1,1,1,36,Female,University,False,False,...,16,13.25,15.582868,"[0.828125, 1.0, 0.96875, 0.96875, 0.640625, 0....",0.828125,0.8125,17.367742,"[10.3176, 19.26643, 14.28441, 15.3505, 8.26959...",NaN,NaN
1,189842-198434,1,1,1,1,25,Female,University,False,False,...,16,16.00,9.153442,"[0.828125, 1.0, 0.96875, 0.96875, 0.640625, 0....",1.000000,1.0000,9.153442,"[8.934598, 8.7303, 15.23631, 10.87219, 6.22189...",NaN,NaN
2,285256-429012,1,1,1,1,27,Male,High School,False,False,...,16,15.50,6.216273,"[0.828125, 1.0, 0.96875, 0.96875, 0.640625, 0....",0.968750,0.9375,6.633005,"[5.776597, 13.51637, 6.810495, 5.625495, 5.892...",NaN,NaN
3,486637-941302,1,1,1,1,27,Male,University,False,False,...,16,15.50,8.996853,"[0.828125, 1.0, 0.96875, 0.96875, 0.640625, 0....",0.968750,0.9375,9.272375,"[7.770398, 11.45151, 7.735799, 6.586198, 10.07...",NaN,NaN
4,522111-792046,1,1,1,1,24,Male,Masters,False,False,...,16,10.25,10.141496,"[0.828125, 1.0, 0.96875, 0.96875, 0.640625, 0....",0.640625,0.4375,10.067041,"[12.67211, 15.00682, 12.10811, 9.307809, 10.55...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,883881-672590,2,2,2,0,19,Male,High School,False,False,...,16,NaN,242.962203,15.18513771,NaN,NaN,1.070318,"[2.0045, 1.3836, 0.6834001, 0.9342001, 0.951, ...",227.0,14.1875
253,901440-351171,2,2,2,0,19,Female,High School,False,False,...,16,NaN,366.414080,22.90088,NaN,NaN,1.246306,"[2.604299, 1.3343, 0.7325999, 0.6843, 0.715799...",294.0,18.3750
254,913584-192754,2,2,2,0,25,Female,Masters,False,False,...,16,NaN,468.229890,29.26436813,NaN,NaN,1.984025,"[4.927498, 2.3661, 1.5031, 1.5705, 1.5157, 1.1...",236.0,14.7500
255,967476-330431,2,2,2,0,25,Male,University,False,False,...,16,NaN,371.182687,23.19891793,NaN,NaN,1.734499,"[6.857299, 2.711, 1.1996, 1.0274, 1.2185, 1.57...",214.0,13.3750


## Cleaning

In [24]:
df = pd.read_csv("DataResults/final.csv")
df_conditions = df.iloc[: , 1:][["Condition", "Starting Interruption", "Control"]]
df_conditions = df_conditions.drop_duplicates()

starting_interruption = []
starting_task = []
conditions = []
metric_description = []
task_description = []
t_exp = []
p_exp = []
difference_exp = []
t_control = []
p_control = []
difference_control = []
t_condition = []
p_condition = []
# entry = [] # condition, metric, task, t-exp, p-exp , difference-exp, t-control, p-control, difference-control

In [25]:
df_conditions

,Condition,Starting Interruption,Control
0,1,1,1
15,2,1,1
60,1,2,1
75,2,2,1
124,1,1,0
139,2,1,0
190,1,2,0
206,2,2,0


In [26]:
# helpers for interpreting t-test statistics

def is_improvement(change):
    if (change > 0):
        return "increased [+]"
    return "decreased [-]"

def is_significant(p):
    if (p <= 0.005):
        return "extremely significantly [**]"
    elif (p <= 0.05):
        return "significantly [*]"
    elif (p <= 0.03):
        return "slightly"
    return ""

def compute_improvement(col1, col2):
    col3 = col2 - col1
    avg = sum(col3)/len(col3)
    t, p = stats.ttest_ind(col1, col2)
    return t, p, avg

In [27]:
def compare_improvement(interruption, task, condition, metric_descript, df_data, pre_metric, post_metric):
    t_0, p_0, diff_0 = compute_improvement(
        df_data[(df_data['Starting Task'] == 1) & (df_data['Condition'] == interruption) & (df_data['Starting Interruption'] == task) & (df_data['Control'] == condition)][pre_metric].astype(float), 
        df_data[(df_data['Starting Task'] == 1) & (df_data['Condition'] == interruption) & (df_data['Starting Interruption'] == task) & (df_data['Control'] == condition)][post_metric].astype(float))
    t_1, p_1, diff_1 = compute_improvement(
        df_data[(df_data['Starting Task'] == 2) & (df_data['Condition'] == interruption) & (df_data['Starting Interruption'] == task) & (df_data['Control'] == condition)][pre_metric].astype(float), 
        df_data[(df_data['Starting Task'] == 2) & (df_data['Condition'] == interruption) & (df_data['Starting Interruption'] == task) & (df_data['Control'] == condition)][post_metric].astype(float))
    diff_experimental = df_data[(df_data['Starting Task'] == 1) & (df_data['Condition'] == interruption) & (df_data['Starting Interruption'] == task) & (df_data['Control'] == condition)][post_metric].astype(float) - df_data[(df_data['Starting Task'] == 1) & (df_data['Condition'] == interruption) & (df_data['Starting Interruption'] == task) & (df_data['Control'] == condition)][pre_metric].astype(float)
    diff_control = df_data[(df_data['Starting Task'] == 2) & (df_data['Condition'] == interruption) & (df_data['Starting Interruption'] == task) & (df_data['Control'] == condition)][post_metric].astype(float) - df_data[(df_data['Starting Task'] == 2) & (df_data['Condition'] == interruption) & (df_data['Starting Interruption'] == task) & (df_data['Control'] == condition)][pre_metric].astype(float)
    t_2, p_2, _ = compute_improvement(diff_experimental, diff_control)

    starting_interruption.append(interruption)
    starting_task.append(task)
    conditions.append(condition)
    metric_description.append(metric_descript)
    t_exp.append(t_0)
    p_exp.append(p_0)
    difference_exp.append(diff_0)
    t_control.append(t_1)
    p_control.append(p_1)
    difference_control.append(diff_1)
    t_condition.append(t_2)
    p_condition.append(p_2)

In [28]:
for index, row in df_conditions.iterrows():
    
    #if row['Starting Task'] == 1:
    #    compare_improvement(row['Condition'], row['Starting Task'], row['Control'], 'accuracy on primary task', df, 'a_p_correctness', 'te_p_correctness')
    #    print("draw")
    #else:
    #    compare_improvement(row['Condition'], row['Starting Task'], row['Control'], 'accuracy on primary task', df, 'a_p_movestotal', 'te_p_movestotal')
    #    print("hanoi")
    compare_improvement(row['Condition'], row['Starting Interruption'], row['Control'], 'accuracy on interrupting task', df, 'a_i_percentage', 'te_i_percentage')
    compare_improvement(row['Condition'], row['Starting Interruption'], row['Control'], 'speed on primary task', df, 'a_p_time', 'te_p_time')
    compare_improvement(row['Condition'], row['Starting Interruption'], row['Control'], 'number of interrupting tasks', df, 'a_i_count', 'te_i_count')

    compare_improvement(row['Condition'], row['Starting Interruption'], row['Control'], 'interruption lag / speed on interrupting task', df, 'a_i_time', 'te_i_time')
    compare_improvement(row['Condition'], row['Starting Interruption'], row['Control'], 'resumption lag', df, 'a_p_resumption', 'te_p_resumption')
    # compare_improvement(row['Starting Interruption'], row['Starting Task'], row['Condition'], 'interruption lag', df, 'a_i_interruptionlag', 'te_i_interruptionlag')
    compare_improvement(row['Condition'], row['Starting Interruption'], row['Control'], 'interruption effort', df, 'a_i_e_effort', 'te_i_e_effort')
    compare_improvement(row['Condition'], row['Starting Interruption'], row['Control'], 'interruption confidence', df, 'a_i_e_confidence', 'te_i_e_confidence')
    compare_improvement(row['Condition'], row['Starting Interruption'], row['Control'], 'primary task effort', df, 'a_p_e_effort', 'te_p_e_effort')
    compare_improvement(row['Condition'], row['Starting Interruption'], row['Control'], 'primary task confidence', df, 'a_p_e_confidence', 'te_p_e_confidence')

In [29]:
columnTitles = {
    "Starting Interruption": starting_interruption,
    "Starting Task": starting_task,
    "Condition": conditions,
    "Metric": metric_description,
    "t (exp)": t_exp,
    "p (exp)": p_exp,
    "diff (exp)": difference_exp,
    "t (control)": t_control,
    "p (control)": p_control,
    "diff (control)": difference_control,
    "t (condition)": t_condition, 
    "p (condition)": p_condition
}

print(len(starting_interruption), len(starting_task), len(conditions), len(metric_description), len(t_exp), len(p_exp), len(difference_exp), 
     len(t_control), len(p_control), len(difference_control))

dataframe = pd.DataFrame(columnTitles)
dataframe.to_csv('tasks-constant.csv')
print('DONE')

72 72 72 72 72 72 72 72 72 72
DONE
